In [1]:
import pandas as pd
import numpy as np
from alive_progress import alive_bar, alive_it 

In [2]:
rankings_df = pd.read_csv("team_rankings.csv")
rankings_df.head()
# rankings_df["date"].info()


,date,url,1_team_name,1_team_id,1_team_ranking_points,2_team_name,2_team_id,2_team_ranking_points,3_team_name,3_team_id,...,27_team_ranking_points,28_team_name,28_team_id,28_team_ranking_points,29_team_name,29_team_id,29_team_ranking_points,30_team_name,30_team_id,30_team_ranking_points
0,2022-03-07,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
1,2022-03-08,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
2,2022-03-09,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
3,2022-03-10,https://www.hltv.org/ranking/teams/2022/march/07,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,endpoint,7234,55,spirit,7020,51,eternal-fire,11251,50
4,2022-02-28,https://www.hltv.org/ranking/teams/2022/februa...,natus-vincere,4608,1000,gambit,6651,744,g2,5995,...,56,apeks,9806,50,skade,10386,49,bluejays,5347,48


In [3]:
matches_df = pd.read_csv("all_matches_stats.csv")
matches_df.head()
# matches_df.info()

C:\Users\alecw\AppData\Local\Temp\ipykernel_17816\3713765491.py:1: DtypeWarning: Columns (20,31,42,53,64,84,95,106,117,128) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_df = pd.read_csv("all_matches_stats.csv")


,match_url,match_date,map,t1_name,t1_id,t1_total_rw,t1_fh_rw,t1_sh_rw,t1_ot_rw,t1_rating,...,t2p5_kills,t2p5_hskills,t2p5_assists,t2p5_fassists,t2p5_deaths,t2p5_kdratio,t2p5_kddiff,t2p5_adr,t2p5_fkdiff,t2p5_rating
0,/stats/matches/mapstatsid/46584/hellraisers-vs...,2017-05-17,train,hellraisers,5310,16,13,3,0,1.66,...,5.0,2.0,1.0,0.0,17.0,33.3%,-12.0,35.9,-3.0,0.26
1,/stats/matches/mapstatsid/62534/envy-vs-north,2018-03-08,cobblestone,north,7533,16,8,8,0,1.02,...,15.0,9.0,1.0,0.0,22.0,63.3%,-7.0,58.5,-3.0,0.78
2,/stats/matches/mapstatsid/61361/fnatic-vs-gambit,2018-02-17,inferno,fnatic,4991,16,10,6,0,1.28,...,7.0,5.0,3.0,1.0,22.0,45.8%,-15.0,49.2,-3.0,0.37
3,/stats/matches/mapstatsid/57159/liquid-vs-cloud9,2017-11-12,overpass,liquid,5973,5,3,2,0,0.89,...,16.0,5.0,3.0,0.0,16.0,71.4%,0.0,67.6,0.0,1.00
4,/stats/matches/mapstatsid/123194/dbl-poney-vs-ago,2021-07-04,ancient,dbl-poney,11003,16,6,10,0,1.04,...,15.0,10.0,2.0,0.0,22.0,56.7%,-7.0,64.6,-1.0,0.81


In [4]:
# now lets add ranking team ranking information to the matches dataset #
# extract the two team name columns
t1_names = matches_df["t1_name"]
t2_names = matches_df["t2_name"]

t1_rankings = []
t2_rankings = []
# with alive_bar(len(matches_df), force_tty=True):
for i in range(len(matches_df)):
    date = matches_df["match_date"].iloc[i]
    rankings_series = rankings_df.loc[rankings_df["date"] == date, :].squeeze()
    t1_rank = int(x.index[0].split("_")[0]) if (x:=rankings_series[rankings_series == t1_names[i]]).size == 1 else 0
    t2_rank = int(x.index[0].split("_")[0]) if (x:=rankings_series[rankings_series == t2_names[i]]).size == 1 else 0
    t1_rankings.append(t1_rank)
    t2_rankings.append(t2_rank)
        # bar()
        
t1_rankings = pd.Series(t1_rankings).replace(0, np.nan)
t2_rankings = pd.Series(t2_rankings).replace(0, np.nan)


KeyboardInterrupt: 

-----

# EDA DATA PREP FOR USE BEFORE MODELS.PY:

In [ ]:

matches_dataset = matches_df.copy()

# t1_rw, t2_rw = matches_dataset["t1_total_rw"], matches_dataset["t2_total_rw"]
winners = []
for index, row in matches_dataset.iterrows():
    winners.append("t1" if row['t1_total_rw'] > row['t2_total_rw'] else "t2")

map_col_index = matches_dataset.columns.get_loc("map")
matches_dataset.insert(map_col_index, "winner", winners, allow_duplicates=False)

round_win_diffs = []
for index, row in matches_dataset.iterrows():
    round_win_diffs.append(row['t1_total_rw'] - row['t2_total_rw'])

winner_col_index = matches_dataset.columns.get_loc("winner")
matches_dataset.insert(winner_col_index + 1, "rw_diff", round_win_diffs, allow_duplicates=False)

t1_rank_col_index = matches_dataset.columns.get_loc("t1_id") 
matches_dataset.insert(t1_rank_col_index + 1, "t1_rank", t1_rankings, allow_duplicates=False)

# get updated index now that column has been inserted prior
t2_rank_col_index = matches_dataset.columns.get_loc("t2_id") 
matches_dataset.insert(t2_rank_col_index + 1, "t2_rank", t2_rankings, allow_duplicates=False)

# unranked teams will be given rank of 31 (one lower than the lowest rank possible)
matches_dataset["t1_rank"].fillna(31, inplace=True)
matches_dataset["t2_rank"].fillna(31, inplace=True)
# replace stray '-' with NaN
matches_dataset.replace("-",None, inplace=True)

# convert kdratio variables to decimal and make sure rating and adr columns are floats
for tn in (1,2):
    matches_dataset[f"t{tn}_rating"] = matches_dataset[f"t{tn}_rating"].astype("float32")
    for pn in (1,2,3,4,5):
        matches_dataset[f"t{tn}p{pn}_kdratio"].fillna("50%",inplace=True)
        try:
            matches_dataset[f"t{tn}p{pn}_kdratio"] = matches_dataset[f"t{tn}p{pn}_kdratio"].apply(
                lambda x: 0.01*float(x.replace("%","")) if x else None)
        except:
            print("fuck")
            
        matches_dataset[f"t{tn}p{pn}_adr"] = matches_dataset[f"t{tn}p{pn}_adr"].astype("float32")
        matches_dataset[f"t{tn}p{pn}_rating"] = matches_dataset[f"t{tn}p{pn}_rating"].astype("float32")
datatypes = list(matches_dataset.dtypes)
columns = list(matches_dataset.columns)
# print({columns[i]:datatypes[i] for i in range(len(columns))})




# force team names to be lowercase
matches_dataset["t1_name"] = matches_dataset["t1_name"].str.lower()
matches_dataset["t2_name"] = matches_dataset["t2_name"].str.lower()

matches_dataset["map"] = matches_dataset["map"].astype("category") 
matches_dataset["t1_name"] = matches_dataset["t1_name"].astype("category")
matches_dataset["t2_name"] = matches_dataset["t2_name"].astype("category")  

print(dict(matches_dataset.dtypes))
matches_dataset.to_csv(index=False, path_or_buf="all_matches_dataset.csv")

{'match_url': dtype('O'), 'match_date': dtype('O'), 'winner': dtype('O'), 'rw_diff': dtype('int64'), 'map': CategoricalDtype(categories=['ancient', 'cache', 'cobblestone', 'dust2', 'inferno',
                  'mirage', 'nuke', 'overpass', 'train', 'tuscan', 'vertigo'],
, ordered=False), 't1_name': CategoricalDtype(categories=['0-o', '00nation', '00prospects', '01', '0to100',
                  '100-thieves', '100pg', '1337', '1337huania', '13th-hour',
                  ...
                  'zerozone', 'zigma', 'zj', 'zoctai', 'zone', 'zooweemama',
                  'zorka', 'ztrewq', 'zxcv', 'zzzzz'],
, ordered=False), 't1_id': dtype('int64'), 't1_rank': dtype('float64'), 't1_total_rw': dtype('int64'), 't1_fh_rw': dtype('int64'), 't1_sh_rw': dtype('int64'), 't1_ot_rw': dtype('int64'), 't1_rating': dtype('float32'), 't1_fkills': dtype('int64'), 't1_cw': dtype('int64'), 't1p1_player': dtype('O'), 't1p1_kills': dtype('int64'), 't1p1_hskills': dtype('int64'), 't1p1_assists': dtype('int64'

In [ ]:
"""
import csv

def format_date(hltv_date : str):
    month, day, year = hltv_date.split("/")
    return "-".join((f"20{year}", month.rjust(2,'0'), day.rjust(2,'0')))

output = []
fieldnames = []

with open("matches_stats.csv","r", encoding="utf-8") as f:
    csvreader = csv.DictReader(f)
    fieldnames = list(dict(list(csvreader)[0]).keys())

with open("matches_stats.csv","r", encoding="utf-8") as f:
    csvreader = csv.DictReader(f)
    # fieldnames = list(dict(list(csvreader)[0]).keys())
    for row in csvreader:
        # row["match_date"] = format_date(row["match_date"])
        output.append(row)
print(len(output))

with open("matches_stats.csv","w", encoding="utf-8", newline="") as f:
    csvwriter = csv.DictWriter(f, fieldnames=fieldnames)
    csvwriter.writeheader()
    csvwriter.writerows(output)

"""

'\nimport csv\n\ndef format_date(hltv_date : str):\n    month, day, year = hltv_date.split("/")\n    return "-".join((f"20{year}", month.rjust(2,\'0\'), day.rjust(2,\'0\')))\n\noutput = []\nfieldnames = []\n\nwith open("matches_stats.csv","r", encoding="utf-8") as f:\n    csvreader = csv.DictReader(f)\n    fieldnames = list(dict(list(csvreader)[0]).keys())\n\nwith open("matches_stats.csv","r", encoding="utf-8") as f:\n    csvreader = csv.DictReader(f)\n    # fieldnames = list(dict(list(csvreader)[0]).keys())\n    for row in csvreader:\n        # row["match_date"] = format_date(row["match_date"])\n        output.append(row)\nprint(len(output))\n\nwith open("matches_stats.csv","w", encoding="utf-8", newline="") as f:\n    csvwriter = csv.DictWriter(f, fieldnames=fieldnames)\n    csvwriter.writeheader()\n    csvwriter.writerows(output)\n\n'

In [ ]:
def get_matches_by_team(team_name : str):
    return matches_dataset.loc[ ((matches_dataset["t1_name"] == team_name.lower()) 
                                | (matches_dataset["t2_name"] == team_name.lower())) , :]

g2_matches_df = get_matches_by_team("G2")
faze_matches_df = get_matches_by_team("FaZe")

print(g2_matches_df.info())
print("")
print(faze_matches_df.info())

g2_matches_df[""].value_counts()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1124 entries, 24 to 62813
Columns: 135 entries, match_url to t2p5_rating
dtypes: category(3), float32(22), float64(34), int64(63), object(13)
memory usage: 1.3+ MB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1093 entries, 16 to 62692
Columns: 135 entries, match_url to t2p5_rating
dtypes: category(3), float32(22), float64(34), int64(63), object(13)
memory usage: 1.3+ MB
None


KeyError: ''